In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Load the data using pandas
data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')

In [2]:

# Initialize the Dash app
app = dash.Dash(__name__)

# List of years
year_list = [i for i in range(1980, 2024, 1)]

# Create the layout of the app
app.layout = html.Div([
    #####################################################################
    # TASK 2.1: Create a Dash application and give it a meaningful title
    #####################################################################
    html.H1("Automobile Statistics Dashboard",
            style={'textAlign': 'center', 'color': '#503D36'}),
    
    
    ######################################################################################
    # TASK 2.2: Add drop-down menus to your dashboard with appropriate titles and options
    ######################################################################################
    html.Div([
    # Dropdown Year or Period
    dcc.Dropdown(id = 'dropdown-statistics', 
                 options=[
                     {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
                     {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
                 ],
                 value = 'Yearly Statistics',
                 placeholder='Select a report type',
                 style={'width':'80%', 'padding':'3px', 'font-size':'20px','textAlign': 'center'}),
    ]),
    
    # Dropdown Year
    html.Div([
    dcc.Dropdown(id='select-year',
                 options=[{'label': i, 'value': i} for i in year_list],
                 value = 1980,
                 placeholder='Select the year',
                 style={'width':'80%', 'padding':'3px', 'font-size':'20px','textAlign': 'center'})
    ]),
    
    #########################################################################################
    # TASK 2.3: Add a division for output display with appropriate id and classname property
    #########################################################################################
    html.Div([
        html.Div(id = 'output-container', 
                 className = 'chart-grid', 
                 #style={'display': 'flex'},
                 #style = {'width': '100%','margin':'auto', 'display': 'inline-block'}
                ),
    ])
    
    
])

#########################################################################################
# TASK 2.4: Creating Callbacks; Define the callback function to update the input 
#           container based on the selected statistics and the output container
#########################################################################################
@app.callback(
    Output(component_id='select-year', component_property='disabled'),
    Input(component_id='dropdown-statistics',component_property='value')
)
def update_input_container(selected_statistics):
    if selected_statistics =='Yearly Statistics': 
        return False
    else: 
        return True


@app.callback(
    Output(component_id='output-container', component_property='children'),
    [Input(component_id='dropdown-statistics', component_property='value'),
     Input(component_id='select-year', component_property='value')
    ])    
def update_output_container(selected_statistics, input_year):
    if selected_statistics == 'Recession Period Statistics':
        # Filter the data for recession periods
        recession_data = data[data['Recession'] == 1]
######################################################################
# TASK 2.5: Create and display graphs for Recession Report Statistics
######################################################################
#Plot 1 Automobile sales fluctuate over Recession Period (year wise) using line chart
         # grouping data for plotting
        yearly_rec=recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        # Plotting the line graph
        R_chart1 = dcc.Graph(
            figure=px.line(yearly_rec, 
                x='Year',
                y='Automobile_Sales',
                title="Automobile sales fluctuate over Recession Period (year wise)"))
#..........
#Plot 2 Calculate the average number of vehicles sold by vehicle type and represent as a Bar chart
#............
        average_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()                           
        R_chart2  = dcc.Graph(
            figure=px.bar(
                average_sales, 
                x= 'Vehicle_Type', 
                y= 'Automobile_Sales', 
                title='Average Number of Vehicles Sold by Type')
        )
        
# Plot 3 : Pie chart for total expenditure share by vehicle type during recessions
            # grouping data for plotting
        exp_rec= recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        R_chart3 = dcc.Graph(
            figure=px.pie(exp_rec, 
                          values='Advertising_Expenditure', 
                          names='Vehicle_Type', 
                          title="Adversiting Expenditure of Vehicle Types during Recession"
                )
        )
#..........
# Plot 4 Develop a Bar chart for the effect of unemployment rate on vehicle type and sales
#............
        type_unemp = recession_data.groupby(['Vehicle_Type', 'unemployment_rate'])['Automobile_Sales'].mean().reset_index()
        R_chart4 = dcc.Graph(
            figure = px.bar(type_unemp, 
                            x='Vehicle_Type', 
                            y='Automobile_Sales',
                            color='unemployment_rate',
                            title='Unemployment relation to Vehicle Type')

            
            )

        return [
            html.Div(className='chart-item', children=[html.Div(children=R_chart1),html.Div(children=R_chart2)]),
            html.Div(className='chart-item', children=[html.Div(children=R_chart3),html.Div(children=R_chart4)])
        ]
    

######################################################################
# TASK 2.6: Create and display graphs for Yearly Report Statistics
######################################################################

    # Yearly Statistic Report Plots                             
    elif (input_year and selected_statistics=='Yearly Statistics') :
        yearly_data = data[data['Year'] == input_year]
                              
#..........
# Plot 1 :Yearly Automobile sales using line chart for the whole period.
#............
        yas = data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart1 = dcc.Graph(
            figure=px.line(yas, 
                           x='Year', 
                           y='Automobile_Sales',
                           title = 'Yearly Automobile sales'))
            
#..........
# Plot 2 :Total Monthly Automobile sales using line chart.
#............
        tmas = yearly_data.groupby('Month')['Automobile_Sales'].sum().reset_index()
        Y_chart2 = dcc.Graph(
            figure=px.line(
                tmas, 
                x= 'Month', 
                y='Automobile_Sales', 
                title= 'Total Monthly Automobile Sales'))
    
# ..........
# Plot 3 bar chart for average number of vehicles sold during the given year
#............
  
        #avr_vdata = yearly_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        avr_vdata = yearly_data.groupby(['Year', 'Vehicle_Type'], as_index = False)['Automobile_Sales'].mean().reset_index()

        Y_chart3 = dcc.Graph(
            figure = px.bar(avr_vdata,
                            x='Vehicle_Type', 
                            y='Automobile_Sales',
                            color='Vehicle_Type',
                            title='Average Vehicles Sold by Vehicle Type in the year {}'.format(input_year))
        )
#..........
# Plot 4 Total Advertisement Expenditure for each vehicle using pie chart
#..........
            
        VTypetotal = yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        Y_chart4 = dcc.Graph(
            figure=px.pie(VTypetotal, 
                          values='Advertising_Expenditure', 
                          names='Vehicle_Type', 
                          title="Adversiting Expenditure of Vehicle Types"
                         )
        )
            
            
        
    
        return[
             html.Div(className='chart-item', children=[html.Div(children=Y_chart1),
                                                        html.Div(children=Y_chart2)]),
             html.Div(className='chart-item', children=[html.Div(children=Y_chart3),
                                                        html.Div(children=Y_chart4)])
         ]
##################################################################################################################################################################################


##################################################################################################################################################################################    
    
    
    
    
    
if __name__ == '__main__':
    app.run_server(mode='inline',port=8040)
    #app.run(debug=False,port=8020)